# Retail Sales Data Warehouse - Exploratory Data Analysis
## EDA Starter Notebook

This notebook provides a starting point for exploratory data analysis of the retail sales data warehouse.

## 1. Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
from pathlib import Path

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path for imports
sys.path.append(str(Path.cwd().parent))

# Import custom modules
from src.etl.extract import DataExtractor
from src.etl.transform import DataTransformer
from src.utils.db_connection import get_connection

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Setup complete!")

## 2. Load Data

Load data from the data warehouse or raw files for analysis.

In [ ]:
# Option 1: Load from CSV files
# extractor = DataExtractor()
# df_sales = extractor.extract_from_csv('sales.csv')

# Option 2: Load from database
# with get_connection('postgresql') as db:
#     query = "SELECT * FROM v_sales_summary LIMIT 10000"
#     df_sales = pd.read_sql_query(query, db.connection)

# For demonstration, create a sample dataset
df_sales = pd.DataFrame({
    'sale_id': range(1, 101),
    'date': pd.date_range('2024-01-01', periods=100, freq='D'),
    'customer_id': np.random.randint(1, 50, 100),
    'product_id': np.random.randint(1, 20, 100),
    'quantity': np.random.randint(1, 10, 100),
    'unit_price': np.random.uniform(10, 100, 100),
    'total_amount': np.random.uniform(10, 500, 100)
})

print(f"Loaded {len(df_sales)} sales records")
df_sales.head()

## 3. Data Overview

In [ ]:
# Display basic information
print("Dataset Shape:", df_sales.shape)
print("\nColumn Data Types:")
print(df_sales.dtypes)
print("\nMissing Values:")
print(df_sales.isnull().sum())

In [ ]:
# Statistical summary
df_sales.describe()

## 4. Data Quality Checks

In [ ]:
# Check for duplicates
duplicates = df_sales.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Check for negative values in numeric columns
numeric_cols = df_sales.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    negative_count = (df_sales[col] < 0).sum()
    if negative_count > 0:
        print(f"Warning: {col} has {negative_count} negative values")

## 5. Exploratory Analysis

### 5.1 Sales Trend Over Time

In [ ]:
# Daily sales trend
if 'date' in df_sales.columns:
    df_sales['date'] = pd.to_datetime(df_sales['date'])
    daily_sales = df_sales.groupby('date')['total_amount'].sum().reset_index()
    
    plt.figure(figsize=(14, 6))
    plt.plot(daily_sales['date'], daily_sales['total_amount'], marker='o', linewidth=2)
    plt.title('Daily Sales Trend', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Total Sales Amount', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 5.2 Sales Distribution

In [ ]:
# Distribution of sales amounts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df_sales['total_amount'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Sales Amount', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Total Amount', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)

# Box plot
axes[1].boxplot(df_sales['total_amount'])
axes[1].set_title('Sales Amount Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Amount', fontsize=12)

plt.tight_layout()
plt.show()

### 5.3 Top Products/Customers Analysis

In [ ]:
# Top 10 products by revenue
if 'product_id' in df_sales.columns:
    top_products = df_sales.groupby('product_id')['total_amount'].sum().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    top_products.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Top 10 Products by Revenue', fontsize=16, fontweight='bold')
    plt.xlabel('Product ID', fontsize=12)
    plt.ylabel('Total Revenue', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Top 10 customers by purchase amount
if 'customer_id' in df_sales.columns:
    top_customers = df_sales.groupby('customer_id')['total_amount'].sum().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    top_customers.plot(kind='bar', color='lightgreen', edgecolor='black')
    plt.title('Top 10 Customers by Purchase Amount', fontsize=16, fontweight='bold')
    plt.xlabel('Customer ID', fontsize=12)
    plt.ylabel('Total Purchase Amount', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

### 5.4 Correlation Analysis

In [ ]:
# Correlation heatmap for numeric columns
numeric_df = df_sales.select_dtypes(include=[np.number])

if len(numeric_df.columns) > 1:
    plt.figure(figsize=(10, 8))
    correlation_matrix = numeric_df.corr()
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Correlation Heatmap', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 6. Key Metrics Summary

In [ ]:
# Calculate key metrics
metrics = {
    'Total Sales': df_sales['total_amount'].sum(),
    'Average Sale Amount': df_sales['total_amount'].mean(),
    'Median Sale Amount': df_sales['total_amount'].median(),
    'Total Transactions': len(df_sales),
    'Unique Customers': df_sales['customer_id'].nunique() if 'customer_id' in df_sales.columns else 'N/A',
    'Unique Products': df_sales['product_id'].nunique() if 'product_id' in df_sales.columns else 'N/A'
}

print("\n=== KEY METRICS ===")
for metric, value in metrics.items():
    if isinstance(value, (int, float)):
        print(f"{metric}: {value:,.2f}")
    else:
        print(f"{metric}: {value}")

## 7. Next Steps

This starter notebook provides a foundation for EDA. Consider extending the analysis with:

1. **Time-based analysis**: Monthly, quarterly, yearly trends
2. **Segmentation**: Customer segments, product categories
3. **Cohort analysis**: Customer retention and lifetime value
4. **Geographic analysis**: Sales by region/store
5. **Predictive modeling**: Sales forecasting, demand prediction
6. **RFM analysis**: Recency, Frequency, Monetary value
7. **Market basket analysis**: Product associations

Happy analyzing! 📊